In [1]:
from sklearn import datasets
diabetes = datasets.load_diabetes(as_frame=True)["frame"]
diabetes

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930,220.0


In [71]:
X = diabetes.iloc[:,:-1]
y = diabetes.iloc[:,-1]

from sklearn.model_selection import train_test_split

X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,random_state=0)

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_tr,y_tr)
print("linear model score:",lm.score(X_te,y_te))

linear model score: 0.33222203269065154


In [72]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.5)
rr.fit(X_tr,y_tr)
print("linear model score:",rr.score(X_te,y_te))
rr.coef_

linear model score: 0.35667773997495866


array([   8.3095978 , -121.63358099,  388.91388623,  219.55009665,
        -19.77469383,  -70.82842908, -183.84763557,  125.0866159 ,
        328.9173479 ,  100.08665295])

In [78]:
from sklearn.linear_model import Lasso
lass = Lasso(alpha=0.2)
lass.fit(X_tr,y_tr)
print("linear model score:",lass.score(X_te,y_te))

linear model score: 0.3433304728856845
[  -0.          -90.57212015  546.28988664  196.86417823   -0.
  -19.04570829 -198.35369805    0.          469.75585901    0.        ]


In [76]:
import numpy as np

idx = np.argsort(np.abs(lass.coef_))  # sort zero to largest
idx = idx[::-1]                       # reverse
X.columns[idx]

Index(['bmi', 's5', 's3', 'bp', 'sex', 's2', 's6', 's4', 's1', 'age'], dtype='object')

In [92]:
from sklearn.model_selection import cross_val_score,KFold
import pandas as pd
kf = KFold(n_splits=8,shuffle=True,random_state=10)

scores = cross_val_score(lm,X,y,cv=kf)
print("scores with all predictors:")
print("mean =",np.median(scores),"  std =",pd.Series(scores).mad())

scores = cross_val_score(lm,X.iloc[:,idx[:5]],y,cv=kf)
print("scores with top 5 predictors:")
print("mean =",np.median(scores),"  std =",pd.Series(scores).mad())



scores with all predictors:
mean = 0.446424589776787   std = 0.10432320819851176
scores with top 5 predictors:
mean = 0.4473418753323214   std = 0.08791888871827909


In [22]:
from sklearn.model_selection import cross_validate,KFold

alpha = [0.02*i for i in range(1,51)]
score = []
kf = KFold(n_splits=5,shuffle=True,random_state=0)
for a in alpha:
    lass = Lasso(alpha=a)
    result = cross_validate(lass,X_tr,y_tr,cv=kf)
    score.append(result["test_score"].mean())
    # lass.fit(X_tr,y_tr)
    # score.append(lass.score(X_te,y_te))

score

[0.5072676626547923,
 0.5073226823566308,
 0.5067791895972422,
 0.5058131157043485,
 0.5039401131635326,
 0.5015400635908068,
 0.4985710663905908,
 0.49539082637961124,
 0.49215538676441356,
 0.4889448865770293,
 0.48570341585778226,
 0.4823885548344042,
 0.4789970799691538,
 0.47575921382492775,
 0.47310073442588435,
 0.4710076022440143,
 0.46869975118490004,
 0.46656807945706874,
 0.46434165529146565,
 0.46212594019801206,
 0.45979657205893537,
 0.4573536965217957,
 0.4547973234418832,
 0.45212745110788416,
 0.44934407946770694,
 0.44644720846757746,
 0.4437171397350722,
 0.4409698245385686,
 0.43811647780304835,
 0.4351565974295083,
 0.4320911920699837,
 0.4289196872874822,
 0.42569017933261366,
 0.4225179893072866,
 0.41924228297393445,
 0.41605736402407134,
 0.4127943038000561,
 0.4094416080637515,
 0.40622688847838617,
 0.40311980513436313,
 0.3999955411103614,
 0.3969122381767146,
 0.39375566897736014,
 0.3905595899580868,
 0.38732956345095426,
 0.3840148700808782,
 0.3806151749

In [20]:
X_te.shape

(89, 10)

In [ ]:
pd.read